In [51]:
import os
import glob
import numpy as np

In [52]:
b = np.load("dice.npy")

In [53]:
print(b)

[[0.9789225  0.44414757 0.55058034 0.50232465 0.58960209 0.67589053
  0.52301345 0.61007986 0.62648957 0.59736167 0.6793211  0.64099502
  0.64438705 0.71106957 0.61203124 0.65369105 0.50190868 0.65345336
  0.55207384 0.50463036 0.4804881  0.49576679 0.43596248 0.66000682
  0.46839481 0.50746807 0.60057514 0.51623172 0.689361   0.50664563
  0.6525412  0.78782998]]


In [44]:
path_img = "/home/user/Documents/dataset/Mindboggle101/mindboggle/image_in_MNI152_normalized"
path_seg = "/home/user/Documents/dataset/Mindboggle101/mindboggle/label_31_reID_merged"
img_file = []
seg_file = [] 
files = glob.glob(os.path.join(path_img,"*.nii.gz"))
for file in files:
    if "flipped" not in file:
        img_file.append(file)
        tmp = os.path.split(file)
        tmp = os.path.join(path_seg,tmp[1])
        seg_file.append(tmp)


In [50]:
list(range(1,61))

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60]